<a href="https://colab.research.google.com/github/AmirRazaMBA/TF_CertificationExam/blob/main/Introduction%20to%20TensorFlow/03_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
import math
import numpy as np
import matplotlib.pyplot as plt
import logging
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

In [6]:
tfds.disable_progress_bar()
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [7]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [30]:
label_names = metadata.features['label'].names
print("Label names: " + str(label_names))

Label names: ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [9]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: " + str(num_train_examples))
print("Number of test examples:     " + str(num_test_examples))

Number of training examples: 60000
Number of test examples:     10000


In [10]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

In [11]:
train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

In [12]:
train_dataset =  train_dataset.cache()  # cache it for better perfromance (optional)
test_dataset  =  test_dataset.cache()

In [13]:
model = Sequential()
model.add(Flatten(input_shape=(28,28,1)))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=10, activation='softmax'))     # multiple categories here so using softmax
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [14]:
BATCH_SIZE = 32
print(BATCH_SIZE)

32


In [15]:
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

In [28]:
train_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [20]:
math.ceil(num_train_examples/BATCH_SIZE)

1875

In [31]:
model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))   # note below are the exact steps in each epoch

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2796 - accuracy: 0.8960
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2653 - accuracy: 0.9026
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2596 - accuracy: 0.9041
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2472 - accuracy: 0.9076
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2362 - accuracy: 0.9104


In [22]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

313/313 [==============================] - 1s 5ms/step - loss: 0.3451 - accuracy: 0.8724
Accuracy on test dataset: 0.8723999857902527


In [32]:
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [33]:
predictions[0]

array([2.38818552e-06, 3.39445130e-08, 6.78808475e-03, 2.53372914e-08,
       9.85369265e-01, 2.16766409e-08, 7.84007553e-03, 4.20542767e-11,
       1.12559995e-07, 2.55567824e-11], dtype=float32)

In [34]:
np.argmax(predictions[0])

4

In [35]:
test_labels[0]

4